In [16]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
#import cv2
#from PIL import Image
#from pybaselines.whittaker import asls
from scipy import interpolate
from scipy.ndimage import gaussian_filter
from scipy.signal import cheby1, filtfilt, find_peaks, peak_widths
from tqdm import tqdm

from preprocessing_functions import (load_frames)

video loading please wait...
dimensions are (18133, 128, 128) as a uint16


In [17]:
video_11_01 = "//files.ubc.ca/team/BNRC/NINC/Raymond Lab/Kaiiiii/cortical data/Pilot/2023_11_01/405700_f3_stage2_pilot_day3_violet.tif"

In [18]:
load_frames(video_11_01);

video loading please wait...
dimensions are (18133, 128, 128) as a uint16
